In [ ]:
# pip install langchain
# pip install langchain-openai  
# pip install langchain-community
# pip install langchain-text-splitters
# pip install PyPDF2
# pip install faiss-cpu
# pip install tiktoken
# pip install python-dotenv

In [1]:
from PyPDF2 import PdfReader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

In [5]:
pdf = PdfReader('D:/AI Projects/Query_Document/Magazine.pdf')

In [6]:
raw_text = ''
for i, page in enumerate(pdf.pages):
    content = page.extract_text()
    if content:
        raw_text += content


In [7]:
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=800,
    chunk_overlap=200,
    length_function=len
)
texts = text_splitter.split_text(raw_text)

In [ ]:
embeddings = OpenAIEmbeddings()

doc_search = FAISS.from_texts(texts, embeddings)

In [10]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

#Question answering function
def ask_question(query):
    docs = doc_search.similarity_search(query, k=3)
    context = "\n\n".join([doc.page_content for doc in docs])
    
    prompt = f"""Based on the following context, answer the question.

Context:
{context}

Question: {query}

Answer:"""
    
    response = llm.invoke(prompt)
    return response.content


query = "How many characters are there in this magazine?"
answer = ask_question(query)
print(answer)

There are several characters mentioned in the context, including Miss Anderson, Mr. Lewis, Mr. Wyndham Lewis, Mr. Eliot, Yeats, Lady Gregory, John Rodker, and Ezra Pound.
